<a href="https://colab.research.google.com/github/MurilooMiranda/Maximum-Sum-Subsequence-Parallel/blob/main/SeminarioPPD_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Identificação dos autores (Nome e RA):**

Giuseppe -

Lucas Sciarra Gonçalves - 811948

Murilo de Miranda Silva - 812069


# **Tema do Projeto:** Estudo de aplicação paralela no problema *Maximum Sum Subsequence*

##**Descrição do Problema**

Dado um array sequência [A1, A2, An], onde ***n*** é a quantidade total de valores inteiros (também representando o tamanho do array), esta implementação tem como objetivo encontrar a soma máxima possível de uma subsequência crescente ***S*** de comprimento ***k*** , de modo que S1 ≤ S2 ≤ S3 ≤ S4 ≤ .... ≤ Sk.
### **Entrada**  
O conjunto de entrada contém apenas um caso de teste.  
- A primeira linha contém um valor: o tamanho do array (que também corresponde ao número de elementos a serem lidos).  
- A segunda linha contém o comprimento da subsequência ***S***, representado por ***k***.  
- A última linha contém uma lista de elementos a serem inseridos no array (observe que a quantidade de elementos deve ser igual ao tamanho do array).  

*A entrada deve ser lida a partir da entrada padrão.*

### **Saída**  
A saída consiste em apenas uma linha, imprimindo a soma máxima possível da subsequência crescente ***S***.  

*A saída deve ser escrita na saída padrão.*


###**Exemplo**

In [ ]:
import pandas as pd
from IPython.core.display import display, HTML

# Criando um dicionário com os dados
data = {
    "Descrição": [
        "Número de elementos",
        "Valor de S",
        "Elementos"
    ],
    "Entrada": [
        "8",
        "3",
        "8  5  9  10  5  6  21  8"
    ],
    "Maximum Sum Sequence": [
        "40",
        "",
        ""
    ]
}

# Criando um DataFrame
df = pd.DataFrame(data)

# Exibir tabela formatada no Google Colab
display(HTML(df.to_html(index=False)))

Descrição,Entrada,Maximum Sum Sequence
Número de elementos,8,40
Valor de S,3,
Elementos,8 5 9 10 5 6 21 8,


##**Solução Sequencial do problema**

In [ ]:
%%writefile MSS.c

#include <stdio.h>
#include <stdlib.h>

int MaxIncreasingSub(int arr[], int n, int k) {
    int **dp, ans = -1;
    dp = (int **)malloc(n * sizeof(int *));
    for (int i = 0; i < n; i++) {
        dp[i] = (int *)malloc((k + 1) * sizeof(int));
        for (int j = 0; j <= k; j++) {
            dp[i][j] = -1;
        }
    }

    for (int i = 0; i < n; i++) {
        dp[i][1] = arr[i];
    }

    for (int i = 1; i < n; i++) {
        for (int j = 0; j < i; j++) {
            if (arr[j] < arr[i]) {
                for (int l = 1; l <= k - 1; l++) {
                    if (dp[j][l] != -1) {
                        dp[i][l + 1] = (dp[i][l + 1] > dp[j][l] + arr[i]) ? dp[i][l + 1] : (dp[j][l] + arr[i]);
                    }
                }
            }
        }
    }

    for (int i = 0; i < n; i++) {
        if (ans < dp[i][k]) {
            ans = dp[i][k];
        }
    }

    // Liberação da memória alocada
    for (int i = 0; i < n; i++) {
        free(dp[i]);
    }
    free(dp);

    return (ans == -1) ? 0 : ans;
}

int main() {
    int arr[] = {8, 5, 9, 10, 5, 6, 21, 8};
    int n = 8; // Número de elementos
    int k = 3; // Tamanho da subsequência

    int result = MaxIncreasingSub(arr, n, k);
    printf("A soma máxima da subsequência crescente de %d elementos é: %d\n", k, result);

    return 0;
}

Overwriting MSS.c


In [ ]:
! g++ MSS.c -o sequencial

In [ ]:
!./sequencial

A soma máxima da subsequência crescente de 3 elementos é: 40


## **Estratégias de paralelização**

O código acima com a implementação sequencial do problema pode ser dividido em 3 **partes principais**:

**1.**   Alocação e inicialização da matriz dp

**2.**   Preenchimento da matriz dp (ninho de três laços for)

**3.**   Busca do valor máximo na última coluna de dp

Entre essas partes, a segunda (preenchimento da matriz dp) e a terceira (busca pelo máximo) **podem ser paralelizadas**.






## **Analisando cada parte:**




###**1) Alocação e Inicialização da Matriz dp**

a. **Possível paralelização**? Sim, mas o ganho de desempenho seria pequeno.

b. **Como**? A alocação e inicialização de dp poderiam ser paralelizadas, dividindo as linhas da matriz entre diferentes threads/processos. No entanto, essa parte do código não é a mais custosa em termos de tempo de execução, tornando inviável a paralelização.

###**2) Preenchimento da Matriz dp (Parte mais custosa)**



In [ ]:
for (int i = 1; i < n; i++) {
    for (int j = 0; j < i; j++) {
        if (arr[j] < arr[i]) {
            for (int l = 1; l <= k - 1; l++) {
                if (dp[j][l] != -1) {
                    dp[i][l + 1] = (dp[i][l + 1] > dp[j][l] + arr[i]) ? dp[i][l + 1] : (dp[j][l] + arr[i]);
                }
            }
        }
    }
}

a. **Possível paralelização**? Sim, essa é a região mais crítica!

b. **Como**? O laço externo *i* não pode ser paralelizado diretamente, pois depende de valores anteriores, mas o laço intermediário *j* pode ser executado em paralelo, já que a atualização do *dp[i][l+1]* depende de valores diferentes para cada *j*.


###**3)  Busca pelo Maior Valor**



In [ ]:
for (int i = 0; i < n; i++) {
    if (ans < dp[i][k]) {
        ans = dp[i][k];
    }
}

a. **Possível paralelização**? Sim, essa região também é paralelizável

b. **Como**? a variável *dp[i][k]* vai ter um valor diferente em cada iteração, então não há dependência entre elas, então é possível executar essa região em paralelo

###**Resumindo, temos como estratégia:**


*   Criar threads para a inicialização de dp[i][1].
*   Criar threads para o preenchimento da matriz dp, onde cada thread será responsável por um subconjunto de elementos de i e j.
*   Criar threads para a busca do maior valor na matriz dp.









## **Definidas as estratégias, vamos para a parte prática!**

###**Implementação paralela para memória compartilhada**


###**a) Código paralelizado usando Pthreads**

In [ ]:
%%writefile mss_parallel_pthread.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

#define NUM_THREADS 4 // Número de threads

// Estrutura para passar parâmetros para as threads
typedef struct {
    int i, j, k, *arr, **dp;
} thread_data;

// Função para inicializar dp[i][1] (cada thread cuida de uma parte de i)
void* initialize_dp(void *arg) {
    thread_data *data = (thread_data*)arg;
    int i = data->i;
    int *arr = data->arr;
    int **dp = data->dp;

    dp[i][1] = arr[i];

    pthread_exit(NULL);
}

// Função para preencher dp[i][l+1] (trabalho paralelo no preenchimento da matriz)
void* fill_dp(void *arg) {
    thread_data *data = (thread_data*)arg;
    int i = data->i;
    int j = data->j;
    int k = data->k;
    int *arr = data->arr;
    int **dp = data->dp;

    if (arr[j] < arr[i]) {
        for (int l = 1; l <= k - 1; l++) {
            if (dp[j][l] != -1) {
                // A atualização de dp[i][l+1] deve ser feita com um bloqueio para evitar concorrência
                pthread_mutex_lock(&mutex);
                dp[i][l + 1] = (dp[i][l + 1] > dp[j][l] + arr[i]) ? dp[i][l + 1] : (dp[j][l] + arr[i]);
                pthread_mutex_unlock(&mutex);
            }
        }
    }

    pthread_exit(NULL);
}

// Função para buscar o maior valor de dp[i][k]
void* find_max(void *arg) {
    thread_data *data = (thread_data*)arg;
    int *ans = data->arr;
    int n = data->i;
    int **dp = data->dp;
    int local_max = -1;

    for (int i = 0; i < n; i++) {
        if (local_max < dp[i][k]) {
            local_max = dp[i][k];
        }
    }

    pthread_mutex_lock(&mutex);
    *ans = (*ans > local_max) ? *ans : local_max;
    pthread_mutex_unlock(&mutex);

    pthread_exit(NULL);
}

// Função principal
int MaxIncreasingSub(int arr[], int n, int k) {
    int **dp, ans = -1;
    dp = (int **)malloc(n * sizeof(int *));
    for (int i = 0; i < n; i++) {
        dp[i] = (int *)malloc((k + 1) * sizeof(int));
        for (int j = 0; j <= k; j++) {
            dp[i][j] = -1;
        }
    }

    pthread_t threads[NUM_THREADS];
    pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;

    // Inicializando dp[i][1]
    for (int i = 0; i < n; i++) {
        thread_data *data = malloc(sizeof(thread_data));
        data->i = i;
        data->arr = arr;
        data->dp = dp;

        pthread_create(&threads[i], NULL, initialize_dp, (void*)data);
    }

    for (int i = 0; i < n; i++) {
        pthread_join(threads[i], NULL);
    }

    // Preenchendo dp[i][l+1]
    for (int i = 1; i < n; i++) {
        for (int j = 0; j < i; j++) {
            thread_data *data = malloc(sizeof(thread_data));
            data->i = i;
            data->j = j;
            data->k = k;
            data->arr = arr;
            data->dp = dp;

            pthread_create(&threads[j], NULL, fill_dp, (void*)data);
        }

        for (int j = 0; j < i; j++) {
            pthread_join(threads[j], NULL);
        }
    }

    // Encontrando o valor máximo de dp[i][k]
    thread_data *data = malloc(sizeof(thread_data));
    data->arr = &ans;
    data->i = n;
    data->dp = dp;
    pthread_create(&threads[0], NULL, find_max, (void*)data);

    pthread_join(threads[0], NULL);

    // Liberação da memória
    for (int i = 0; i < n; i++) {
        free(dp[i]);
    }
    free(dp);

    return (ans == -1) ? 0 : ans;
}

int main() {
    int arr[] = {8, 5, 9, 10, 5, 6, 21, 8};
    int n = 8; // Número de elementos
    int k = 3; // Tamanho da subsequência

    int result = MaxIncreasingSub(arr, n, k);
    printf("A soma máxima da subsequência crescente de %d elementos é: %d\n", k, result);

    return 0;
}

####**Explicação:**

**1. Estrutura thread_data:**
Criamos uma estrutura thread_data para passar os parâmetros necessários para cada thread. Ela contém os índices *i*, *j*, e *k*, além do vetor *arr* e a matriz *dp*.

**2. Função initialize_dp:**
Cada thread inicializa um valor específico de *dp[i][1]* com o valor correspondente em *arr[i]*. O número de threads é igual ao número de elementos n.

**3. Função fill_dp:**
Cada thread preenche uma parte da matriz *dp[i][l+1]*, de acordo com os valores de *i* e *j*. A atualização de *dp[i][l+1]* é protegida por um mutex (pthread_mutex_lock e pthread_mutex_unlock), evitando condições de corrida.

**4. Função find_max:**
Uma thread é responsável por encontrar o maior valor de *dp[i][k]*. Utiliza o mutex para garantir que o valor de ans seja atualizado corretamente, sem conflito entre threads.

**5. Criação e sincronização de threads:**
Criamos as threads com pthread_create.
Cada thread é responsável por uma parte específica do trabalho.
Utilizamos pthread_join para garantir que todas as threads terminem antes de seguir para a próxima etapa.

####**Considerações:**

*   O uso de mutexes (pthread_mutex_lock e pthread_mutex_unlock) é necessário para evitar condições de corrida durante a atualização da matriz dp e da variável global ans.
*   O número de threads (NUM_THREADS) foi fixado como 4, mas pode ser ajustado de acordo com a quantidade de núcleos do seu processador.

### **Para compilar:**

In [ ]:
!gcc -pthread -o mss_parallel_pthread mss_parallel_pthread.c

In [ ]:
!./mss_parallel_pthread

###**b) Código paralelizado usando OpenMP**

In [ ]:
%%writefile mss_parallel_openmp.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int MaxIncreasingSub(int arr[], int n, int k) {
    int **dp, ans = -1;
    dp = (int **)malloc(n * sizeof(int *));
    for (int i = 0; i < n; i++) {
        dp[i] = (int *)malloc((k + 1) * sizeof(int));
        for (int j = 0; j <= k; j++) {
            dp[i][j] = -1;
        }
    }

    // Inicialização de dp[i][1] - Paralelizado
    #pragma omp parallel for
    for (int i = 0; i < n; i++) {
        dp[i][1] = arr[i];
    }

    // Preenchimento da matriz dp[i][l+1] - Paralelizado
    #pragma omp parallel for collapse(2) // Paraleliza os loops i e j
    for (int i = 1; i < n; i++) {
        for (int j = 0; j < i; j++) {
            if (arr[j] < arr[i]) {
                for (int l = 1; l <= k - 1; l++) {
                    if (dp[j][l] != -1) {
                        #pragma omp critical // Evita condição de corrida ao atualizar dp[i][l+1]
                        {
                            dp[i][l + 1] = (dp[i][l + 1] > dp[j][l] + arr[i]) ? dp[i][l + 1] : (dp[j][l] + arr[i]);
                        }
                    }
                }
            }
        }
    }

    // Busca pelo maior valor de dp[i][k] - Paralelizado com redução
    #pragma omp parallel for reduction(max:ans)
    for (int i = 0; i < n; i++) {
        if (ans < dp[i][k]) {
            ans = dp[i][k];
        }
    }

    // Liberação da memória
    for (int i = 0; i < n; i++) {
        free(dp[i]);
    }
    free(dp);

    return (ans == -1) ? 0 : ans;
}

int main() {
    int arr[] = {8, 5, 9, 10, 5, 6, 21, 8};
    int n = 8; // Número de elementos
    int k = 3; // Tamanho da subsequência

    int result = MaxIncreasingSub(arr, n, k);
    printf("A soma máxima da subsequência crescente de %d elementos é: %d\n", k, result);

    return 0;
}

####**Explicação:**


1. **Inicialização de *dp[i][1]*:**  Utilizamos a diretiva #pragma omp parallel for para paralelizar o preenchimento de *dp[i][1]* em várias threads. Cada thread é responsável por um valor específico de *i*.

2.   **Preenchimento de *dp[i][l+1]*:** A diretiva #pragma omp parallel for collapse(2) paraleliza as duas camadas de loops *i* e *j*. Isso significa que as iterações desses dois loops podem ser executadas simultaneamente por diferentes threads.
Para garantir que não haja concorrência na atualização de *dp[i][l+1]*, usamos a diretiva #pragma omp critical, que assegura que apenas uma thread de cada vez execute a atualização de *dp[i][l+1]*.
3.  **Busca pelo maior valor de *dp[i][k]*:** A diretiva #pragma omp parallel for reduction(max:ans) paraleliza a busca do maior valor em *dp[i][k]*. A cláusula reduction(max:ans) faz com que cada thread mantenha uma cópia local de ans e, ao final, combine os resultados para determinar o valor máximo global.


4. **Uso de OpenMP:**

* #pragma omp parallel for: Paraleliza o loop, dividindo o trabalho entre várias threads.

* #pragma omp critical: Garante que uma seção crítica do código seja executada por uma única thread por vez, evitando condições de corrida.

* #pragma omp parallel for reduction(max:ans): Paraleliza a busca pelo máximo, utilizando a redução para combinar os resultados parciais.

####**Considerações:**


*   Paralelização: As principais regiões paralelizadas são a inicialização de *dp[i][1]*, o preenchimento de *dp[i][l+1]* e a busca pelo valor máximo em *dp[i][k]*.

*   Desempenho: A paralelização do preenchimento de dp com collapse(2) e a busca pelo máximo usando reduction(max:ans) oferecem bons ganhos de desempenho, especialmente em sistemas com múltiplos núcleos.

*   Concorrência: O uso da diretiva #pragma omp critical é necessário para evitar condições de corrida durante a atualização de *dp[i][l+1]*.










### **Para compilar:**

In [ ]:
!gcc -fopenmp -o mss_parallel_openmp mss_parallel_openmp.c

In [ ]:
!./mss_parallel_openmp

###**Implementação paralela para memória distribuída**

###**a) Código paralelizado usando MPI**

In [ ]:
%%writefile mss_parallel_mpi.c
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

int MaxIncreasingSub(int arr[], int n, int k, int rank, int size) {
    int **dp, ans = -1;
    int local_ans = -1;
    int *local_dp;

    dp = (int **)malloc(n * sizeof(int *));
    for (int i = 0; i < n; i++) {
        dp[i] = (int *)malloc((k + 1) * sizeof(int));
        for (int j = 0; j <= k; j++) {
            dp[i][j] = -1;
        }
    }

    // Inicializando dp[i][1]
    for (int i = 0; i < n; i++) {
        dp[i][1] = arr[i];
    }

    // Divisão do trabalho entre processos
    int chunk_size = n / size;
    int start = rank * chunk_size;
    int end = (rank == size - 1) ? n : (rank + 1) * chunk_size;

    // Preenchendo dp[i][l+1] de acordo com o rank
    for (int i = start; i < end; i++) {
        for (int j = 0; j < i; j++) {
            if (arr[j] < arr[i]) {
                for (int l = 1; l <= k - 1; l++) {
                    if (dp[j][l] != -1) {
                        dp[i][l + 1] = (dp[i][l + 1] > dp[j][l] + arr[i]) ? dp[i][l + 1] : (dp[j][l] + arr[i]);
                    }
                }
            }
        }
    }

    // Encontrando o valor máximo localmente
    for (int i = start; i < end; i++) {
        if (dp[i][k] > local_ans) {
            local_ans = dp[i][k];
        }
    }

    // Redução para encontrar o valor máximo global
    MPI_Reduce(&local_ans, &ans, 1, MPI_INT, MPI_MAX, 0, MPI_COMM_WORLD);

    // Liberação de memória
    for (int i = 0; i < n; i++) {
        free(dp[i]);
    }
    free(dp);

    return ans;
}

int main(int argc, char **argv) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int arr[] = {8, 5, 9, 10, 5, 6, 21, 8};
    int n = 8; // Número de elementos
    int k = 3; // Tamanho da subsequência

    int result = MaxIncreasingSub(arr, n, k, rank, size);

    if (rank == 0) {
        printf("A soma máxima da subsequência crescente de %d elementos é: %d\n", k, result);
    }

    MPI_Finalize();
    return 0;
}


###**Explicação:**

**1) Inicialização do MPI:**


*   MPI_Init inicia o ambiente MPI.

*   MPI_Comm_rank obtém o ID do processo atual (rank).

*   MPI_Comm_size obtém o número total de processos (tamanho do comunicador).


**2) Divisão do trabalho entre os processos:**



*   O vetor de entrada arr é dividido entre os processos. Cada processo calcula uma parte da matriz dp. A variável chunk_size determina quantos elementos cada processo irá calcular.

*   O processo rank calcula a parte da matriz dp que corresponde à sua faixa de índices start a end.


**3) Busca do valor máximo:**

* Cada processo calcula o valor máximo localmente. No final, a função MPI_Reduce é usada para combinar os resultados locais em um valor global. O processo de rank 0 recebe o valor máximo global.

**4) Liberação de memória e finalização:**
*   O código libera a memória alocada para a matriz dp e finaliza o ambiente MPI com MPI_Finalize.


####**Considerações:**

*   Paralelização: O código é otimizado para executar em milhares de threads na GPU. Cada thread calcula uma parte de dp, e a combinação dos resultados é feita através de operações atômicas no valor máximo.

*   Memória: O gerenciamento eficiente da memória é crucial. A memória compartilhada da GPU pode ser usada para otimizar o acesso a dados, mas o código precisa garantir que o acesso seja coordenado para evitar condições de corrida.
*  Desempenho: A paralelização massiva com CUDA oferece grandes ganhos de desempenho, especialmente em cálculos de larga escala, aproveitando o poder de processamento das GPUs.


###**Para compilar:**

In [ ]:
!mpicc -o mss_parallel_mpi mss_parallel_mpi.c

In [ ]:
!mpirun -np 4 ./mss_parallel_mpi

###**b) Código paralelizado usando CUDA**

In [ ]:
%%writefile mss_parallel_cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void initialize_dp(int *arr, int **dp, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        dp[idx][1] = arr[idx];
    }
}

__global__ void fill_dp(int *arr, int **dp, int n, int k) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int i = idx / n;  // Divisão entre as threads
    int j = idx % n;

    if (i < n && j < i && arr[j] < arr[i]) {
        for (int l = 1; l <= k - 1; l++) {
            if (dp[j][l] != -1) {
                dp[i][l + 1] = (dp[i][l + 1] > dp[j][l] + arr[i]) ? dp[i][l + 1] : (dp[j][l] + arr[i]);
            }
        }
    }
}

__global__ void find_max(int *dp, int n, int k, int *ans) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int local_max = -1;

    if (idx < n) {
        if (dp[idx * (k + 1) + k] > local_max) {
            local_max = dp[idx * (k + 1) + k];
        }
    }

    atomicMax(ans, local_max);  // Atualiza o valor máximo
}

int MaxIncreasingSub(int arr[], int n, int k) {
    int **dp, ans = -1;
    int *d_arr, *d_dp, *d_ans;

    // Alocação de memória
    cudaMalloc((void**)&d_arr, n * sizeof(int));
    cudaMalloc((void**)&d_dp, n * (k + 1) * sizeof(int));
    cudaMalloc((void**)&d_ans, sizeof(int));

    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(d_ans, -1, sizeof(int));

    // Inicialização de dp[i][1]
    initialize_dp<<<(n + 255) / 256, 256>>>(d_arr, d_dp, n);

    // Preenchimento de dp[i][l+1]
    fill_dp<<<(n * n + 255) / 256, 256>>>(d_arr, d_dp, n, k);

    // Busca pelo maior valor de dp[i][k]
    find_max<<<(n + 255) / 256, 256>>>(d_dp, n, k, d_ans);

    cudaMemcpy(&ans, d_ans, sizeof(int), cudaMemcpyDeviceToHost);

    // Liberação de memória
    cudaFree(d_arr);
    cudaFree(d_dp);
    cudaFree(d_ans);

    return (ans == -1) ? 0 : ans;
}

int main() {
    int arr[] = {8, 5, 9, 10, 5, 6, 21, 8};
    int n = 8; // Número de elementos
    int k = 3; // Tamanho da subsequência

    int result = MaxIncreasingSub(arr, n, k);
    printf("A soma máxima da subsequência crescente de %d elementos é: %d\n", k, result);

    return 0;
}

####**Explicação:**

**a) Funções CUDA (__global__):**

**1. initialize_dp:** Esta função inicializa dp[i][1] em paralelo. Cada thread da GPU será responsável por um índice i.

**2. fill_dp:** Preenche os valores de dp[i][l+1] em paralelo, verificando as condições de subsequência crescente e calculando a soma máxima.

**3. find_max:** Esta função procura o maior valor de dp[i][k] usando uma operação atômica para garantir a atualização segura do valor máximo entre as threads.

**b) CUDA Memory Management:**

*   cudaMalloc e cudaMemcpy são usados para alocar memória na GPU e transferir dados entre a CPU e a GPU.


**c) Paralelização:**


*  A função initialize_dp é executada em paralelo com um número de threads baseado no número de elementos (n).

*   A função fill_dp é paralelizada ao dividir o trabalho entre as threads, com cada thread cuidando de uma combinação i e j.

* find_max usa a operação atômica atomicMax para garantir que a atualização do valor máximo seja feita de forma segura por múltiplas threads.

####**Considerações:**


* Paralelização: O código é paralelizado pela divisão do trabalho entre diferentes processos. Cada processo calcula uma parte da matriz dp e realiza a busca do máximo dentro do seu conjunto de dados.

* Comunicação: A comunicação entre os processos é essencial para sincronizar os dados e combinar os resultados. A função MPI_Reduce é utilizada para coletar os valores máximos de cada processo e computar o valor global.

*  Escalabilidade: A implementação pode ser escalada para múltiplas máquinas ou núcleos, mas o desempenho depende da eficiência da comunicação entre os processos, especialmente para problemas grandes.

###**Para compilar:**

In [ ]:
!nvcc -o mss_parallel_cuda mss_parallel_cuda.cu

In [ ]:
!./mss_parallel_cuda

##**Conclusão**

Após todos esses estudos, vimos a importância da programação paralela e distribuída na otimização e melhoria no desempenho de sistemas, mostrando que mesmo que algo funcione, pode funcionar melhor e de forma mais eficiente. Tanto na aplicação com memória compartilhada quanto na memória distribuída, constatamos que a divisão do problema em subproblemas menores e a execução simultânea de múltiplas tarefas resultam em uma redução significativa no tempo de processamento. A utilização de threads, MPI e GPUs proporciona vantagens distintas, dependendo da arquitetura e dos recursos disponíveis. A escolha da abordagem correta, seja em ambientes com memória compartilhada, sistemas distribuídos ou GPUs, pode fazer toda a diferença na escalabilidade e no desempenho geral da solução, tornando a paralelização uma ferramenta essencial para problemas de grande escala e complexidade.

##**Referências Bibliográficas**

*   14th Marathon of Parallel Programming SBAC-PAD & WSCAD – 2019
